<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Using TensorFlow Keras, Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn.

This is your "Hello World!" of TensorFlow.

### Example TensorFlow Starter Code

```python 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(3, activation='sigmoid', input_dim=2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])

results = model.fit(X,y, epochs=100)

```

### Additional Written Tasks:
1. Investigate the various [loss functions](https://www.tensorflow.org/api_docs/python/tf/keras/losses). Which is best suited for the task at hand (predicting 1 / 0) and why? 
2. What is the difference between a loss function and a metric? Why might we need both in Keras? 
3. Investigate the various [optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers). Stochastic Gradient Descent (`sgd`) is not the learning algorithm dejour anyone. Why is that? What do newer optimizers such as `adam` have to offer? 

In [1]:
import pandas as pd

data = {
    'x1': (0, 0, 1, 0, 1, 1, 0),
    'x2': (0, 1, 0, 1, 0, 1, 0),
    'x3': (1, 1, 1, 0, 0, 1, 0),
    'y' : (0, 1, 1, 1, 1, 0, 0)
}

df = pd.DataFrame.from_dict(data).astype('int')

X = df.drop('y', axis=1).values
y = df['y'].values

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(4, activation='sigmoid', input_dim=3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['binary_accuracy'])

In [32]:
results = model.fit(X,y, epochs=100, verbose=1)

Train on 7 samples
Epoch 1/100
7/7 [==============================] - 0s 48ms/sample - loss: 0.7518 - binary_accuracy: 0.5714
Epoch 2/100
7/7 [==============================] - 0s 429us/sample - loss: 0.7512 - binary_accuracy: 0.5714
Epoch 3/100
7/7 [==============================] - 0s 428us/sample - loss: 0.7507 - binary_accuracy: 0.5714
Epoch 4/100
7/7 [==============================] - 0s 571us/sample - loss: 0.7502 - binary_accuracy: 0.5714
Epoch 5/100
7/7 [==============================] - 0s 571us/sample - loss: 0.7496 - binary_accuracy: 0.5714
Epoch 6/100
7/7 [==============================] - 0s 708us/sample - loss: 0.7491 - binary_accuracy: 0.5714
Epoch 7/100
7/7 [==============================] - 0s 429us/sample - loss: 0.7486 - binary_accuracy: 0.5714
Epoch 8/100
7/7 [==============================] - 0s 429us/sample - loss: 0.7481 - binary_accuracy: 0.5714
Epoch 9/100
7/7 [==============================] - 0s 571us/sample - loss: 0.7476 - binary_accuracy: 0.5714
Epoch 10/1

Epoch 77/100
7/7 [==============================] - 0s 571us/sample - loss: 0.7216 - binary_accuracy: 0.5714
Epoch 78/100
7/7 [==============================] - 0s 286us/sample - loss: 0.7213 - binary_accuracy: 0.5714
Epoch 79/100
7/7 [==============================] - 0s 572us/sample - loss: 0.7210 - binary_accuracy: 0.5714
Epoch 80/100
7/7 [==============================] - 0s 571us/sample - loss: 0.7208 - binary_accuracy: 0.5714
Epoch 81/100
7/7 [==============================] - 0s 571us/sample - loss: 0.7205 - binary_accuracy: 0.5714
Epoch 82/100
7/7 [==============================] - 0s 429us/sample - loss: 0.7202 - binary_accuracy: 0.5714
Epoch 83/100
7/7 [==============================] - 0s 429us/sample - loss: 0.7200 - binary_accuracy: 0.5714
Epoch 84/100
7/7 [==============================] - 0s 429us/sample - loss: 0.7197 - binary_accuracy: 0.5714
Epoch 85/100
7/7 [==============================] - 0s 571us/sample - loss: 0.7194 - binary_accuracy: 0.5714
Epoch 86/100
7/7 [=

In [68]:
maxloss = (0,)
for optimizer in ['adadelta', 'adagrad', 'adam', 'adamax', 'ftrl', 'nadam', 'rmsprop', 'sgd']:
    for metric in ['accuracy', 'binary_accuracy', 'binary_crossentropy', 'hinge',
                   'poisson', 'squared_hinge']:
        for loss in ['binary_crossentropy', 'poisson', 'reduction', 'hinge', 'squared_hinge']:
            model = Sequential([
                Dense(4, activation='sigmoid', input_dim=3),
                Dense(1, activation='sigmoid')
            ])
            model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[metric])
            results = model.fit(X,y, epochs=10, verbose=0)
            if max(results.history[metric]) - min(results.history[metric]) > maxloss[0]:
                maxloss = (max(results.history[metric]) - min(results.history[metric]), max(results.history[metric]), optimizer, metric, loss)

In [69]:
maxloss

(0.2857143, 0.71428573, 'ftrl', 'binary_accuracy', 'poisson')

In [88]:
model = Sequential([
    Dense(4, activation='relu', input_dim=3),
    Dense(1, activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
results = model.fit(X,y, epochs=10, verbose=1)

Train on 7 samples
Epoch 1/10
7/7 [==============================] - 1s 167ms/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 2/10
7/7 [==============================] - 0s 714us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 3/10
7/7 [==============================] - 0s 857us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 4/10
7/7 [==============================] - 0s 572us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 5/10
7/7 [==============================] - 0s 572us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 6/10
7/7 [==============================] - 0s 572us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 7/10
7/7 [==============================] - 0s 570us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 8/10
7/7 [==============================] - 0s 714us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 9/10
7/7 [==============================] - 0s 858us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 10/10
7/7 [=

In [90]:
import numpy as np
model.predict([[0, 1, 1],
               [1, 0, 0],
               [1, 1, 0],
               [0, 0, 1]])

array([[1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

### Build a Tensor Keras Perceptron

Try to match the architecture we used on Monday - inputs nodes and one output node. Apply this architecture to the XOR-ish dataset above. 

After fitting your model answer these questions: 

Are you able to achieve the same results as a bigger architecture from the first part of the assignment? Why is this disparity the case? What properties of the XOR dataset would cause this disparity? 

Now extrapolate this behavior on a much larger dataset in terms of features. What kind of architecture decisions could we make to avoid the problems the XOR dataset presents at scale? 

*Note:* The bias term is baked in by default in the Dense layer.

In [94]:
model = Sequential([
    Dense(2000, activation='relu', input_dim=3),
    Dense(1, activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
results = model.fit(X,y, epochs=10, verbose=1)

Train on 7 samples
Epoch 1/10
7/7 [==============================] - 1s 135ms/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 2/10
7/7 [==============================] - 0s 683us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 3/10
7/7 [==============================] - 0s 714us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 4/10
7/7 [==============================] - 0s 569us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 5/10
7/7 [==============================] - 0s 690us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 6/10
7/7 [==============================] - 0s 714us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 7/10
7/7 [==============================] - 0s 574us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 8/10
7/7 [==============================] - 0s 571us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 9/10
7/7 [==============================] - 0s 427us/sample - loss: 6.5354 - binary_accuracy: 0.5714
Epoch 10/10
7/7 [=

## Try building/training a more complex MLP on a bigger dataset.

Use TensorFlow Keras & the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the canonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the Internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [95]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [96]:
# input image dimensions
img_rows, img_cols = 28, 28

In [97]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [98]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [99]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simplify the problem for now: Zero or all else.

In [100]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [101]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [105]:
model = Sequential([
    Dense(10, activation='sigmoid', input_dim=784),
    Dense(10, activation='sigmoid'),
    Dense(10, activation='sigmoid'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [107]:
from tensorflow.keras.callbacks import EarlyStopping

stop = EarlyStopping(monitor='val_accuracy',
                     min_delta=0.01,
                     patience=3)

In [108]:
model.fit(x_train, y_train,
          epochs=50,
          verbose=1,
          validation_data=(x_test,y_test),
          callbacks=[stop])

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 5s 82us/sample - loss: 0.0104 - accuracy: 0.9971 - val_loss: 0.0135 - val_accuracy: 0.9968
Epoch 2/50
60000/60000 [==============================] - 5s 78us/sample - loss: 0.0087 - accuracy: 0.9977 - val_loss: 0.0142 - val_accuracy: 0.9967
Epoch 3/50
60000/60000 [==============================] - 5s 78us/sample - loss: 0.0074 - accuracy: 0.9980 - val_loss: 0.0134 - val_accuracy: 0.9968
Epoch 4/50
60000/60000 [==============================] - 5s 77us/sample - loss: 0.0062 - accuracy: 0.9983 - val_loss: 0.0172 - val_accuracy: 0.9959


## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?